In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
class EpochLogger(CallbackAny2Vec):
  '''Callback to log information about training'''
  def __init__(self):
    self.epoch = 0
  
  def on_epoch_begin(self, model):
    print("Epoch #{} start".format(self.epoch))
    
  def on_epoch_end(self, model):
    print("Epoch #{} end".format(self.epoch))
    self.epoch += 1
epoch_logger = EpochLogger()
deepwalk = Word2Vec.load("/content/gdrive/My Drive/Task3/deepwalkwithoutmesh.model")
deepvocab= deepwalk.wv.vocab

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
deepwalk.similar_by_word('back')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('10003978.0', 0.8104172945022583),
 ('10024891.0', 0.807929277420044),
 ('10003988.0', 0.7950490117073059),
 ('lower', 0.7649259567260742),
 ('10024890.0', 0.7393198609352112),
 ('low', 0.7236460447311401),
 ('10000424.0', 0.7200608849525452),
 ('neck', 0.7181693911552429),
 ('10038194.0', 0.6881814002990723),
 ('10028836.0', 0.6857696771621704)]

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
!pip install tweet-preprocessor
!pip install autocorrect
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION)
from textblob import Word, TextBlob
import string
import re
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')


def lemma(text):
  s=""
  for x in text.split():
    if(x=="rt"):
      continue
    else:
      u=Word(x)
      b = TextBlob(u.lemmatize())
      z=str(b.correct())
      s=s+z+" "
  return s.strip()
#print(emoji_pattern.sub(r'', text)) # no emoji



def process(text):
  text = p.clean(str(text))
  text= re.sub(r'\s([@][\w_-]+)', '', text, flags=re.MULTILINE)
  text = ''.join([t for t in text if t not in string.punctuation])
  x=text.lower()
  text=x
  li=stopwords.words('english')
  li.append("feel")
  li.append("feels")
  li.append("feeling")
  li.append("like")
  text = [t for t in text.split() if t not in li]
  #b = TextBlob(x)
  #text=str(b.correct())
  x=""
  for z in text:
    x=x+z+" "
  text=lemma(x.strip())
  return text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import pandas as pd
test=pd.read_csv('/content/drive/My Drive/task3_validation.tsv',delimiter='\t',encoding='utf-8')

In [0]:
test = test.filter(['tweet_id','begin','end','type','extraction','drug','tweet','meddra_code','meddra_term'])
test = test[test['meddra_term'].notna()]

In [0]:
test

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,allergies,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,HURT YOUR Liver,avelox,"@ashleylvivian if #avelox has hurt your liver,...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,AD,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,cipro,pt of mine died from cipro rt @ciproispoison: ...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
390,342283136480206848,110.0,122.0,ADR,hypochondria,baclofen,"hmm, interesting. i may have been undergoing b...",10020965.0,hypochondriasis
391,342283136480206848,55.0,65.0,ADR,withdrawal,baclofen,"hmm, interesting. i may have been undergoing b...",10048010.0,withdrawal syndrome
392,342299048105091072,11.0,14.0,ADR,low,vyvanse,i hate the low after the high on vyvanse,10024919.0,low mood
393,342310086363983873,34.0,45.0,ADR,weight gain,tysabri,on a related note: tysabri causes weight gain....,10047896.0,weight gain


In [0]:
test['extraction'] = test['extraction'].apply(process)

In [0]:
test['meddra_code'] = test['meddra_code'].apply(lambda x: str(x))

In [0]:
test['drug'] = test['drug'].apply(process)

In [0]:
test

,tweet_id,begin,end,type,extraction,drug,tweet,meddra_code,meddra_term
0,332317478170546176,28.0,37.0,ADR,clergy,avelox,"do you have any medication allergies? ""asthma!...",10013661.0,drug allergy
1,347806215776116737,31.0,46.0,ADR,hurt liver,avelox,"@ashleylvivian if #avelox has hurt your liver,...",10024668.0,liver damage
2,350336129817509888,48.0,50.0,ADR,ad,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003731.0,attention deficit disorder
3,350336129817509888,88.0,93.0,ADR,focus,baclofen,"apparently, baclofen greatly exacerbates the ""...",10003738.0,attention impaired
4,332540699692130304,11.0,15.0,ADR,died,micro,pt of mine died from cipro rt @ciproispoison: ...,10011906.0,death
...,...,...,...,...,...,...,...,...,...
390,342283136480206848,110.0,122.0,ADR,hypochondria,baclofen,"hmm, interesting. i may have been undergoing b...",10020965.0,hypochondriasis
391,342283136480206848,55.0,65.0,ADR,withdrawal,baclofen,"hmm, interesting. i may have been undergoing b...",10048010.0,withdrawal syndrome
392,342299048105091072,11.0,14.0,ADR,low,vyvanse,i hate the low after the high on vyvanse,10024919.0,low mood
393,342310086363983873,34.0,45.0,ADR,weight gain,tysabri,on a related note: tysabri causes weight gain....,10047896.0,weight gain


In [0]:
test_extraction=test['extraction'].to_list()
test_drugs=test['drug'].to_list()
true_med_codes=test['meddra_code'].to_list()

In [0]:
"weight gain".split()

['weight', 'gain']

In [0]:
possible_med_codes=list(set(true_med_codes))


In [0]:
def closest(words, pos_med_codes, model):
  code=""
  val=0
  for med_code in pos_med_codes:
    if(med_code not in deepvocab):
      continue
    temp = model.n_similarity(words, [med_code])
    if(val < temp):
      code=med_code
      val=temp
  return code

In [0]:
from gensim.models.keyedvectors import KeyedVectors
bioword=KeyedVectors.load_word2vec_format('/content/drive/My Drive/Task3/BioWordVec_PubMed_MIMICIII_d200.vec.bin', limit=200000, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


FileNotFoundError: ignored

In [0]:
bioword.similarity("pain","gain")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.36394724

In [0]:
bioword_vocab= list(bioword.vocab.keys())

In [0]:
common_vocab=list(set(bioword_vocab) & set(deepvocab))

In [0]:
def bioword_closest(term):
  until_now_similar_word=""
  similar_by=0
  for g in  common_vocab:
    temp=bioword.similarity(term,g)
    if(temp>similar_by):
      similar_by=temp
      until_now_similar_word=g
  return until_now_similar_word

In [0]:
#possible_med_codes.remove('10028045.0')
pred_med_codes=[]
for i in range(len(test_extraction)):
  word_set=[]
  for token in test_extraction[i].split():

    if(token in deepvocab):
       word_set.append(token)
    elif(token in common_vocab):
      word_set.append(bioword_closest(token))

    if(test_drugs[i] in deepvocab):
      word_set.append(test_drugs[i])
    elif(test_drugs[i] in common_vocab):
      word_set.append(bioword_closest(test_drugs[i]))

  if(len(word_set)==0):
    pred_med_codes.append("10028045.0")
  else:
    pred_med_codes.append(closest(word_set,possible_med_codes, deepwalk))
  #print(word_set)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
import numpy as np

from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(np.array(true_med_codes), np.array(pred_med_codes), average='micro')

(0.29863013698630136, 0.29863013698630136, 0.29863013698630136, None)

In [0]:
bioword.similar_by_word("pain")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('discomfort/pain', 0.8563158512115479),
 ('pain-', 0.8393102288246155),
 ('discomfort', 0.8343413472175598),
 ('pain/discomfort', 0.832956075668335),
 ('backpain', 0.831273078918457),
 ('pains', 0.8138630390167236),
 ('non-pain', 0.7960410118103027),
 ('pain/', 0.7923266887664795),
 ('discomforts', 0.7886637449264526),
 ('pain/anxiety', 0.7881842851638794)]